In [1]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time
from sklearn.metrics.pairwise import cosine_similarity

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'amazon-beauty/large_merged_data.csv')
print(f'Data path: {data_path}')

# # zero shot save path
# ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/CF_predictions_zero_shot.csv')
# print(f'Zero shot save path: {ZERO_SHOT_SAVE_PATH}')

# # few shot save path
# FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/CF_1_test_predictions_few_shot.csv')
# print(f'Few shot save path: {FEW_SHOT_1_OBS_SAVE_PATH}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/large_merged_data.csv


In [3]:
# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9767 entries, 0 to 9766
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rating          9767 non-null   float64
 1   verified        9767 non-null   bool   
 2   reviewTime      9767 non-null   object 
 3   reviewerID      9767 non-null   object 
 4   asin            9767 non-null   object 
 5   reviewerName    9767 non-null   object 
 6   reviewText      9759 non-null   object 
 7   summary         9759 non-null   object 
 8   unixReviewTime  9767 non-null   object 
 9   vote            1487 non-null   object 
 10  style           6768 non-null   object 
 11  category        9767 non-null   object 
 12  tech1           2 non-null      object 
 13  description     9767 non-null   object 
 14  fit             0 non-null      float64
 15  title           9767 non-null   object 
 16  also_buy        9767 non-null   object 
 17  tech2           0 non-null      f

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
0,1.0,False,2015-08-25,A2RYSCZOPEXOCQ,9790787006,The Cat Next Door,"I use a lot of perfume, I go through a new bot...",This is not going to be my favorite scent.,2015-08-25,NaN,...,NaN,Jenna Jameson,[],298.0,"['B00357FTX8', 'B01NBID7FJ', 'B0017JT658']","{'Shipping Weight:': '12.8 ounces (', 'ASIN: '...",All Beauty,NaN,NaN,13.85
1,5.0,False,2001-06-08,A141OPVE376YFI,B000050B65,Paul G.,"First, a little background. I've switched bet...","Finally, a razor that lives up to the ads",2001-06-08,81,...,NaN,Norelco,[],2.0,"['B01B1O9DOM', 'B00JITDVD2', 'B01KXV16DK', 'B0...",{},All Beauty,NaN,NaN,NaN
2,5.0,False,2001-06-08,A141OPVE376YFI,B000050B65,Paul G.,"First, a little background. I've switched bet...","Finally, a razor that lives up to the ads",2001-06-08,81,...,NaN,Norelco,[],2.0,"['B01B1O9DOM', 'B00JITDVD2', 'B01KXV16DK', 'B0...",{},All Beauty,NaN,NaN,NaN
3,5.0,True,2008-07-25,A1TVTDKNMSQ7XU,B000050B6B,Grandpa Pipes,I've had many Norelco razors in my 50 years of...,Just like new.....,2008-07-25,NaN,...,NaN,Philips Norelco,[],148.0,"['B001IA0PCY', 'B00196W5S4', 'B004URZADG', 'B0...",{'\n Product Dimensions: \n ': '5.1 x 0....,All Beauty,NaN,NaN,64.50
4,5.0,True,2008-07-25,A1TVTDKNMSQ7XU,B000050B6B,Grandpa Pipes,I've had many Norelco razors in my 50 years of...,Just like new.....,2008-07-25,NaN,...,NaN,Philips Norelco,[],148.0,"['B001IA0PCY', 'B00196W5S4', 'B004URZADG', 'B0...",{'\n Product Dimensions: \n ': '5.1 x 0....,All Beauty,NaN,NaN,64.50


In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# Check if Target User Has Rated All Items
def check_unrated_items(user_item_matrix, user_id):
    return user_item_matrix.loc[user_id].isna().sum()


# Example function to create a user-item matrix
def create_user_item_matrix(data, user_column_name='reviewerID', item_column_name='title', rating_column_name='rating'):
    return pd.pivot_table(data, index=user_column_name, columns=item_column_name, values=rating_column_name)

# Example function for item-based collaborative filtering
def item_based_cf(user_item_matrix, target_user_id):
    # Fill NaN values with 0 (indicating no interaction)
    filled_user_item_matrix = user_item_matrix.fillna(0)

    # Transpose matrix to get item-user matrix
    item_user_matrix = filled_user_item_matrix.T

    # Compute item-item similarity matrix
    item_similarity = cosine_similarity(item_user_matrix, dense_output=True)

    # Convert to DataFrame
    item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

    # Get items rated by the target user
    target_user_ratings = filled_user_item_matrix.loc[target_user_id].dropna()

    # Compute weighted ratings
    weighted_ratings = item_similarity_df.loc[target_user_ratings.index].mul(target_user_ratings.values, axis=0)

    # Calculate recommendations
    recommendations = weighted_ratings.sum(axis=0).div(item_similarity_df.loc[target_user_ratings.index].sum(axis=0))
    recommendations = recommendations.drop(target_user_ratings.index, errors='ignore')  # Remove already rated items
    
    # Inside your item_based_cf function, add:
    print(f"Item Similarity DataFrame: {item_similarity_df.head()}")
    print(f"Target User Ratings: {target_user_ratings}")
    print(f"Weighted Ratings: {weighted_ratings.head()}")

    # Return top N recommendations
    return recommendations.nlargest(5)


# Example usage
user_item_matrix = create_user_item_matrix(data)
user_item_matrix

title,"AquaTica Organic Botanical Night Cream with Caviar Extract, Reishi and CoQ10, 2oz",Beauticontrol Show of Hands Instant Manicure 10 Oz.,Colgate Fluoride Toothpaste Strawberry Smash Liquid Gel 4.60 oz (Pack of 6),"CoverGirl Clean Fragrance Free Pressed Powder, Sof","Crest Whitestrips Daily Whitening and Tartar Protection, 38-Count Pouches",Gillette Clinical Advance Solid Sport Triumph 1.7 Ounce,"Hair Rules Curly Whip Styling, 8 oz",IsaDora Colormatic Lip Contour Lipliner #84 Rosewood,"JASON Purifying Tea Tree Nail Saver, 0.5 oz. (Pack of 4) (Packaging May Vary)",Medicool&#39;s MED2191 Turbo File 2 Professional Electric Nail Filing System,...,"niceEshop(TM) Professional Plastic Spray Bottle Water Plant Hair Art Beauty Salon Supply,Clear","niceEshop(TM) Punk Rock Devil Skeleton Hand Claw Hair Clip (Multicolor,1pc)","niceEshop(TM) Round Acrylic Nail Art False Tips Display for Wheel Board Polish Tool (White,Set of 10)",niceeshop(TM) Metallic Mirror Metal T-show Stage Style Women Circle Cuff Elastic Hair Band Ponytail Holder-Gold by niceeshop,tarte lippie lingerie matte tint- whisper (rosy beige),"wet n wild Mega Last Lip Color, Don't Blink Pink, 0.11 Ounce","wet n wild Megalast Lip Color, In the Flesh, 0.11 Fluid Ounce","wet n wild Megalast Lip Color, Mocha-licious, 0.11 Fluid Ounce","wet n wild Megalast Lip Color, Red Velvet, 0.11 Fluid Ounce","wet n wild Silk Finish Lip Stick, Hot Red, 0.13 Ounce"
reviewerID,,,,,,,,,,,,,,,,,,,,,
A105A034ZG9EHO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10JB7YPWZGRF4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10M2MLE2R0L6K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10OYW0QYN13GL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10P0NAKKRYKTZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZMAOC6QC0WEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZPI1JA9XKV8P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZQZIAWSFBHLW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
unrated_items_count = check_unrated_items(user_item_matrix, 'ANV9L0JU6BNL')
print(f'Number of unrated items by user ANV9L0JU6BNL: {unrated_items_count}')
recommendations = item_based_cf(user_item_matrix, 'ANV9L0JU6BNL')
recommendations

Number of unrated items by user ANV9L0JU6BNL: 1865
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Item Similarity DataFrame: title                                                AquaTica Organic Botanical Night Cream with Caviar Extract, Reishi and CoQ10, 2oz   \
title                                                                                                                                     
 AquaTica Organic Botanical Night Cream with Ca...                                                1.0                                     
 Beauticontrol Show of Hands Instant Manicure 1...                                                0.0                                     
 Colgate Fluoride Toothpaste Strawberry Smash L...                                                0.0               

Series([], dtype: float64)